In [13]:
# -----------------------------------------------------------------------------
# Copyright (c) 2022, Lucid Vision Labs, Inc.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
# OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# -----------------------------------------------------------------------------

In [14]:
import time

from arena_api.callback import callback, callback_function
from arena_api.system import system

In [15]:
'''
Callbacks: On Event
    This example demonstrates configuring a callback with events. Events are a
    subset of nodes that invoke callbacks through the underlying events engine.
    The events engine is first initialized to listen for events, then the
    callback is registered using the timestamp test event node. The event is
    generated, along with any data generated from the event. The example then
    waits for the event to process in order to invoke the callback. Registered
    callbacks must also be deregistered before deinitializing the events engine
    in order to avoid memory leaks.
'''

'\nCallbacks: On Event\n    This example demonstrates configuring a callback with events. Events are a\n    subset of nodes that invoke callbacks through the underlying events engine.\n    The events engine is first initialized to listen for events, then the\n    callback is registered using the timestamp test event node. The event is\n    generated, along with any data generated from the event. The example then\n    waits for the event to process in order to invoke the callback. Registered\n    callbacks must also be deregistered before deinitializing the events engine\n    in order to avoid memory leaks.\n'

In [16]:
'''
Once registered, this function is called when the callback event is triggered.
'''

@callback_function.node.on_update
def print_node_value(node, *args, **kwargs):

    print(f'Message from callback')
    print(f'\'{node.name}\' event has triggered this callback')


In [17]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


Created 1 device(s)


In [18]:
# Store initial values, to restore later
streamBufferHandlingMode_initial = \
    device.tl_stream_nodemap['StreamBufferHandlingMode'].value
triggerSource_initial = device.nodemap['TriggerSource'].value
triggerSelector_initial = device.nodemap['TriggerSelector'].value
triggerMode_initial = device.nodemap['TriggerMode'].value


In [19]:
'''
Register callback function on event node  
'''

event_node = device.nodemap.get_node('EventExposureEnd')
handle = callback.register(event_node, print_node_value)
print(f"Registered '{print_node_value.__name__}' function "
      f"in '{event_node.name}' node")


Registered 'print_node_value' function in 'EventExposureEnd' node


In [20]:
'''
Enable trigger mode before setting the source and selector
    and before starting the stream. Trigger mode cannot be turned
    on and off while the device is streaming.
'''

# Make sure Trigger Mode set to 'Off' after finishing this example
device.nodemap.get_node('TriggerMode').value = 'On'

'''
Set the trigger source to software in order to trigger buffers
    without the use of any additional hardware.
    Lines of the GPIO can also be used to trigger.
'''
device.nodemap.get_node('TriggerSource').value = 'Software'

device.nodemap.get_node('TriggerSelector').value = 'FrameStart'

device.tl_stream_nodemap.get_node(
    'StreamBufferHandlingMode').value = 'OldestFirst'

'''
Initialize events
    Turn event notification on
    Select the event type to be notified about
'''
device.initialize_events()
device.nodemap.get_node('EventSelector').value = 'ExposureEnd'
device.nodemap.get_node('EventNotification').value = 'On'


Message from callback
'EventExposureEnd' event has triggered this callback


In [21]:
'''
Register callback function on event node
      Allows for manual triggering
'''

event_node = device.nodemap.get_node('EventExposureEnd')
handle = callback.register(event_node, print_node_value)
print(f"Registered '{print_node_value.__name__}' function "
      f"in '{event_node.name}' node")


Registered 'print_node_value' function in 'EventExposureEnd' node


In [22]:
device.start_stream()
for _ in range(10):

    '''
    Continually check until trigger is armed. Once the trigger is
        armed, it is ready to be executed.
    '''
    while not device.nodemap.get_node('TriggerArmed').value:
        continue

    '''
    Trigger an image buffer manually, since trigger mode is enabled.
        This triggers the camera to acquire a single image buffer.
        A buffer is then filled and moved to the output queue, where
        it will wait to be retrieved.
        Before the image buffer is sent, the exposure end event will
        occur. This will happen on every iteration
    '''
    device.nodemap.get_node('TriggerSoftware').execute()

    '''
    Wait on the event to process it, invoking the registered callback.
        The data is created from the event generation, not from waiting
        on it. If the exposure time is long a Timeout exception may occur
        unless large timeout value is passed to the 'Device.wait_on_event()'
    '''
    device.wait_on_event()

Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'Ev

In [23]:
# Callbacks must be unregistered and events must be deinitialized
callback.deregister(handle)
device.deinitialize_events()

Message from callback
'EventExposureEnd' event has triggered this callback
Message from callback
'EventExposureEnd' event has triggered this callback


In [24]:
# Stop stream before destroying device to set nodes back to initial values
device.stop_stream()

device.nodemap['TriggerSource'].value = triggerSource_initial
device.nodemap['TriggerSelector'].value = triggerSelector_initial
device.nodemap['TriggerMode'].value = triggerMode_initial

# Destroy device (optional)
system.destroy_device()
print('Destroyed all created devices')
